In [2]:
import sys
import geemap
import ee

ee.Initialize()
ee.Authenticate()
geemap.ee_initialize()

#import functions from RivWidthCloud
#NOTE: had to download it locally; couldn't get external .js modules to work
module_path = r'C:\Users\<username>\CursorProjects\testproject\jupyter\RivWidthCloud_Python'
import RivWidthCloud_Python.functions_centerline_width as clFunctions
import RivWidthCloud_Python.functions_river as riverFunctions

In [3]:
#test reach from A2.4 River Morphology (Rio Madre de Dios)
aoi = ee.Geometry.Polygon([
    [
        [-67.18414102495456, -11.402427204567534],
        [-66.57886300981784, -11.402427204567534],
        [-66.57886300981784, -11.09095257894929],
        [-67.18414102495456, -11.09095257894929],
        [-67.18414102495456, -11.402427204567534]
    ]
])
sword = ee.FeatureCollection('projects/gee-book/assets/A2-4/SWORD')

#UTM projection from coords
def get_utm_proj(lon, lat):
    utm_code = ee.Number(lon).add(180).divide(6).ceil().int()
    output = ee.Algorithms.If(
        ee.Number(lat).gte(0),
        ee.String('EPSG:326').cat(utm_code.format('%02d')),
        ee.String('EPSG:327').cat(utm_code.format('%02d'))
    )
    return output

#coords for the reach centroid
coords = aoi.centroid(30).coordinates()
lon = coords.get(0)
lat = coords.get(1)
crs = get_utm_proj(lon, lat)
scale = ee.Number(30)

#reproject image to crs and scale
def rpj(image):
    return image.reproject(crs = crs.getInfo(), scale = scale.getInfo())

#initialize map for rendering
Map = geemap.Map()

In [7]:
##IMPORT AND VISUALIZE SURFACE WATER MASK.
#sample surface water dataset (A2.4 uses v1.3 instead of v1.4)
jrc_yearly = ee.ImageCollection('JRC/GSW1_4/YearlyHistory')

#prune centerline to remove spurious branches
MAXDISTANCE_BRANCH_REMOVAL = 500

#map setup; background optional
Map.centerObject(aoi)
#Map.addLayer(ee.Image.constant(0), {'min': 0, 'palette': ['black']}, 'background', True)

#seasonal and permanent pixels image representing years 1990-2020 in 5yr increments
#aoi layers for 1990 - 2020
for year in range(1990, 2020, 5):
    ##REMOVE NOISE AND SMALL ISLANDS TO SIMPLIFY TOPOLOGY.
    watermask = jrc_yearly.filter(ee.Filter.eq('year', year)).first().gte(2).unmask(0).clip(aoi)

    #image closure operation to fill small holes
    watermask = watermask.focal_max().focal_min()

    #identify small bars and fill them in to create a filled water mask.
    MIN_SIZE = 2000
    barPolys = watermask.Not().selfMask().reduceToVectors(
        geometry = aoi,
        scale = 30,
        eightConnected = True,
    ).filter(ee.Filter.lte('count', MIN_SIZE))  #get small polys
    filled = watermask.paint(barPolys, 1)

    ##IDENTIFYING RIVERS FROM OTHER TYPES OF WATER BODIES.
    #cumulative cost mapping to find pixels connected to a reference centerline
    costmap = filled.Not().cumulativeCost(
        source = watermask.And(ee.Image().toByte().paint(sword, 1)),
        maxDistance = 3000,
        geodeticDistance = False
    )

    rivermask = costmap.eq(0).rename('riverMask')
    channelmask = rivermask.And(watermask)

    #calculate distance from shoreline using distance transform
    distance = clFunctions.CalcDistanceMap(rivermask, 256, scale)

    #calculate gradient of the distance raster (2nd approach)
    gradient = clFunctions.CalcGradientMap(distance, 2, scale)

    #threshold the gradient raster and derive 1px width centerline using skeletonization
    centerlineRaw = clFunctions.CalcOnePixelWidthCenterline(rivermask, gradient, 0.9)
    raw1pxCenterline = rpj(centerlineRaw.eq(1).selfMask())

    #pruning centerline
    cl1px = clFunctions.CleanCenterline(centerlineRaw, MAXDISTANCE_BRANCH_REMOVAL, True)
    cl1px = clFunctions.CleanCenterline(cl1px, MAXDISTANCE_BRANCH_REMOVAL, False)
    final1pxCenterline = rpj(cl1px).eq(1).selfMask()

    #vectorize centerline
    vectorPoly = final1pxCenterline.reduceToVectors(
        geometryType='polygon',
        scale=30,
        crs=crs.getInfo(),
        geometry=aoi,
        eightConnected = True,
    )

    #Map.addLayer(rpj(rivermask), {}, f'rivermask {year}', True)
    Map.addLayer(vectorPoly, {}, f'centerline {year}', True)

Map.addLayerControl()
Map

Map(bottom=557442.0, center=[-11.207366028807511, -66.82743072509767], controls=(WidgetControl(options=['posit…